In [6]:
from moviepy import VideoFileClip
from moviepy.decorators import convert_parameter_to_seconds, convert_path_to_string
import pandas as pd
import scipy.optimize
import numpy as np
import cv2
import subprocess

In [7]:
# COMMON PARAMETERS
start_time = ["00:09:15", "00:19:54","00:28:05", "00:35:20", "00:43:22", '00:54:00',"01:01:21", "01:08:50", "01:17:11", "01:25:43","01:35:01","01:44:26","01:52:34"]
end_time = ["00:17:49","00:26:52","00:34:04","00:40:22","00:49:32","01:00:16","01:07:32","01:15:17","01:23:30","01:32:06","01:41:20","01:50:46","01:58:49"]
input_path = "/Users/goal0312/Desktop/thesis/1_videos/*.mp4"
output_path = "/Users/goal0312/Desktop/thesis/1_videos/chunks/"

In [8]:
time = convert_parameter_to_seconds(("start_time","end_time"))

1. CALCULATE BEST DURATION OF A SMALL CHUNK TO KEEP THE LEAST REMAINDER

In [9]:
# convert timeframes into seconds and divided into minutes
start_scalar = np.array((pd.to_timedelta(start_time).total_seconds())+4)
end_scalar = np.array(pd.to_timedelta(end_time).total_seconds())

# calculate how long each big chunk is
each_big_chunk = np.array(end_scalar - start_scalar)
each_big_chunk, start_scalar, end_scalar

(array([510., 414., 355., 298., 366., 372., 367., 383., 375., 379., 375.,
        376., 371.]),
 array([ 559., 1198., 1689., 2124., 2606., 3244., 3685., 4134., 4635.,
        5147., 5705., 6270., 6758.]),
 array([1069., 1612., 2044., 2422., 2972., 3616., 4052., 4517., 5010.,
        5526., 6080., 6646., 7129.]))

In [10]:
# calculate the bestx for the least remainder
fn = lambda x: sum(each_big_chunk%x)
xMin = 120
xMax = 180

bestx = scipy.optimize.minimize_scalar(fun = fn, bounds=[xMin, xMax]) 
bestx # an object
np.floor(bestx.x)

np.float64(169.0)

In [11]:
# with that bestx, how many small chunks do we get out per each big chunk
numbers_of_small_chunks_from_big_chunks = np.floor((each_big_chunk)/(bestx.x))
sum(numbers_of_small_chunks_from_big_chunks)

np.float64(26.0)

2. CUT VIDEOS INTO CHUNKS

I have 3 videos.

The first frames at each video are 27777, 27775, 27776.
There are 13 starting points, corresponding to 13 chunks.

start_time = ["00:09:15:27", "00:19:54:25","00:28:05:25", "00:35:20:34", "00:43:22:32", '00:54:00:48',"01:01:21:19", "01:08:50:19", "01:17:11:42", "01:25:43:23","01:35:01:42","01:44:26:37","01:52:34:44"];

end_time = ["00:17:49","00:26:52","00:34:04","00:40:22","00:49:32","01:00:16","01:07:32","01:15:17","01:23:30","01:32:06","01:41:20","01:50:46","01:58:49"];


I want to write a function that

    (1) convert start_time from the first frame of each video 

    (2) cut each video in chunks from start_time & end_time to frame

    (3) cut the big start-end chunk into smaller chunks of      3     2     2     1     2     2     2     2     2     2     2     2     2

    (4) repeat for 3 videos

In [12]:
# 1. moviepy: ~7 mins x 13 x 3

# optimal duration in seconds
x = bestx.x*60

tri_start = [start_scalar[0]]
tri_end = [end_scalar[0]]
tri_start, tri_end

# test for the first big chunk
clip = VideoFileClip("05nov_CAM_01_V2.mp4")
for i in range(len(tri_start)):
    cut = clip.subclipped(tri_start[i], tri_end[i])  
    numbers_of_chunks = numbers_of_small_chunks_from_big_chunks[i] #eg first video n0 = 3
    j = 0
    while j < numbers_of_chunks:
        second_cut = cut.subclipped(j*x,(j+1)*x)
        j = j+1
        second_cut.write_videofile(f"big_chunk_{i+1}_small_chunk_{j}.mp4", codec="libx264", audio_codec="aac")

ValueError: end_time (10200.00) should be smaller or equal to the clip's duration (510.00).

In [ ]:
# write into a function
def cut_with_moviepy():
    """
    inputs: video name, starting & ending time for each big chunk, duration of a small chunk, number of chunk
    """

In [ ]:
# 2. ffmpeg


In [ ]:
# not re-decoding: get to the nearest keyframe

input_video = "05nov_CAM_01_V2.mp4"
duration = bestx.x # in seconds

for i in range(len(start_scalar)):
    numbers_of_chunks = numbers_of_small_chunks_from_big_chunks[i]
    j = 0
    while j < numbers_of_chunks:
        output_file = f"chunk_{i+1}_{j+1}.mp4"
        command = [
            "ffmpeg",
            "-i", input_video,
            "-ss", str(start_scalar[i] + j*duration),
            "-to", str(start_scalar[i] + (j+1)*duration),
            "-c", "copy",      # no re-encoding
            output_file
        ]
        subprocess.run(command)
        j += 1


ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --en

In [ ]:
# re-decoding: take longer than moviepy
input_video = "05nov_CAM_01_V2.mp4"
duration = bestx.x # in seconds
fps = 50

for i in range(len(start_scalar)):
    numbers_of_chunks = numbers_of_small_chunks_from_big_chunks[i]
    j = 0
    while j < numbers_of_chunks:
        start_time = start_scalar[i] + j * duration
        end_time = start_scalar[i] + (j + 1) * duration

        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)

        output_file = f"chunk_{i+1}_{j+1}.mp4"

        command = [
            "ffmpeg",
            "-i", input_video,
            "-vf", f"trim=start_frame={start_frame}:end_frame={end_frame},setpts=PTS-STARTPTS",
            "-af", f"atrim=start={start_time}:end={end_time},asetpts=PTS-STARTPTS",
            "-c:v", "libx264",
            "-preset", "fast",
            "-crf", "18",
            "-c:a", "aac",
            "-b:a", "192k",
            output_file
        ]
        subprocess.run(command)
        j += 1

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --en

KeyboardInterrupt: 